# Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, BatchNormalization, LeakyReLU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.datasets import mnist

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

plt.style.use("dark_background")

tf.__version__

'2.17.0'

In [ ]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


(60000, 28, 28)

* See the data and flatten.

In [ ]:
N, H, W = X_train.shape
D = H * W
X_train = X_train.reshape(-1, D)
X_test = X_test.reshape(-1, D)
X_train.shape

(60000, 784)

In [ ]:
latent_dim = 100

X_train = X_train / 255 * 2 - 1
X_test = X_test / 255 * 2 - 1

In [ ]:
def build_generator(latent_dim):
    i = Input(shape=(latent_dim,))
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(i)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(1024, activation=LeakyReLU(alpha=0.2))(x)
    x = BatchNormalization(momentum=0.7)(x)
    x = Dense(D, activation="tanh")(x) # since we normalized layer btwn the interval -1 and 1

    model = Model(i, x)
    return model

In [ ]:
def build_discriminator(img_size):
    i = Input(shape=(img_size,))
    x = Dense(512, activation=LeakyReLU(alpha=0.2))(i)
    x = Dense(256, activation=LeakyReLU(alpha=0.2))(x)
    x = Dense(1, activation="sigmoid")(x)

    model = Model(i, x)
    return model

In [ ]:
discriminator = build_discriminator(D)
discriminator.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                      loss="binary_crossentropy",
                      metrics=["acc"])

# since we can't just train generator by itself, we should write it by an exception
generator = build_generator(latent_dim)

z = Input(shape=(latent_dim,))
generated_img = generator(z)
discriminator.trainable = False
fake_pred = discriminator(generated_img)
combined_model = Model(z, fake_pred)
combined_model.compile(optimizer=Adam(learning_rate=0.0002, beta_1=0.5),
                      loss="binary_crossentropy")

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


In [ ]:
batch_size = 32
epochs = 30000
sample_size = 200

ones = np.ones(batch_size)
zeros = np.zeros(batch_size)

d_losses = []
g_losses = []

if not os.path.exists("gan_images"):
    os.makedirs("gan_images")

In [ ]:
def sample_images(epoch):
    rows, cols = (5, 5)
    noise = np.random.randn(rows*cols, latent_dim)
    imgs = generator.predict(noise)

    imgs = imgs * 0.5 - 0.5

    fig, axs = plt.subplots(rows, cols)
    idx = 0
    for i in range(rows):
      for j in range(cols):
        axs[i, j].imshow(imgs[idx].reshape(H, W), cmap="gray")
        axs[i, j].axis("off")
        idx += 1

    fig.savefig("gan_images/%d.png" % epoch)
    plt.close()

In [ ]:
for epoch in range(epochs):
    N = X_train.shape[0]

    # Selecting random batch number sized of images
    idx = np.random.randint(0, N, batch_size)
    real_imgs = X_train[idx]

    # generating random batch number sized of images (32 iamge)
    noise = np.random.randn(batch_size, latent_dim)
    fake_imgs = generator(noise)

    # discriminator losses and accs
    d_loss_real, d_acc_real = discriminator.train_on_batch(real_imgs, ones)
    d_loss_fake, d_acc_fake = discriminator.train_on_batch(fake_imgs, zeros)
    d_loss = 0.5 * (d_loss_real + d_loss_fake)
    d_acc = 0.5 * (d_acc_real + d_acc_fake)

    # generator losses and accs
    noise = np.random.randn(batch_size, latent_dim)
    g_loss = combined_model.train_on_batch(noise, ones) # we lied by saying fake images are real (we picked ones for noies)

    d_losses.append(d_loss)
    g_losses.append(g_loss)

    if epoch % 100 == 0:
        print(f"epoch: {epoch}/{epochs} -------> d_loss: {d_loss} & d_acc: {d_acc} _______ g_loss:{g_loss[-1]}")

    if epoch % sample_size == 0:
        sample_images(epoch)

epoch: 0/30000 -------> d_loss: 2.112969398498535 & d_acc: 0.4800196886062622 _______ g_loss:0.4797077775001526
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
epoch: 100/30000 -------> d_loss: 2.2753567695617676 & d_acc: 0.478081613779068 _______ g_loss:0.4778350591659546
epoch: 200/30000 -------> d_loss: 2.4082608222961426 & d_acc: 0.47742098569869995 _______ g_loss:0.4772168695926666
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
epoch: 300/30000 -------> d_loss: 2.5214991569519043 & d_acc: 0.47681641578674316 _______ g_loss:0.47664234042167664
epoch: 400/30000 -------> d_loss: 2.6183972358703613 & d_acc: 0.47616681456565857 _______ g_loss:0.4760151207447052
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
epoch: 500/30000 -------> d_loss: 2.7025656700134277 & d_acc: 0.47584068775177 _______ g_loss:0.47570621967315674
epoch: 600/30000 -------> d_loss: 2.7786922454833984 & d_acc: 0.47578704357147217 _______ g_loss:0.47566625475883484
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
epoch: 700/30000 -------> d_loss: 2.848198

In [ ]:
N = X_train.shape[0]
np.random.randint(0, N, batch_size)

In [ ]:
(5.14234 * 100) // 10